### Set up

In [1]:
##https://github.com/zlatankr/Projects/tree/master/Tanzania

In [2]:
import pandas as pd
import numpy as np

from IPython.core.debugger import set_trace

import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#import feature_process_helper
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

from sklearn.metrics import make_scorer

from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Data

In [4]:
data_dir = os.getcwd()+'/data/'

train_data = pd.read_csv(f'{data_dir}train.csv' , index_col = 'new_ids') ##
test_data = pd.read_csv(f'{data_dir}test.csv' , index_col = 'new_ids') ## 



X_train = train_data.copy()
X_test = test_data.copy()
y_train = X_train[['defective']]
X_train.drop(columns=['defective'] , inplace=True)

print(X_train.shape)
print(X_test.shape)

(50000, 39)
(5083, 39)


In [5]:
y_train['defective'] = y_train['defective'].apply(lambda x: 1 if x=='yes' else 0)

/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


###  Feature engineering

In [6]:
def dates(X_train, X_test):
    """
    date_recorded: this might be a useful variable for this analysis, although the year itself would be useless in a practical scenario moving into the future. We will convert this column into a datetime, and we will also create 'year_recorded' and 'month_recorded' columns just in case those levels prove to be useful. A visual inspection of both casts significant doubt on that possibility, but we'll proceed for now. We will delete date_recorded itself, since random forest cannot accept datetime
    """
    for i in [X_train, X_test]:
        i['date_recorded'] = pd.to_datetime(i['date_recorded'])
        i['year_recorded'] = i['date_recorded'].apply(lambda x: x.year)
        i['month_recorded'] = i['date_recorded'].apply(lambda x: x.month)
        i['date_recorded'] = (pd.to_datetime(i['date_recorded'])).apply(lambda x: x.toordinal())
    return X_train, X_test

def dates2(X_train, X_test):
    """
    Turn year_recorded and month_recorded into dummy variables
    """
    for z in ['month_recorded', 'year_recorded']:
        X_train[z] = X_train[z].apply(lambda x: str(x))
        X_test[z] = X_test[z].apply(lambda x: str(x))
        good_cols = [z+'_'+i for i in X_train[z].unique() if i in X_test[z].unique()]
        X_train = pd.concat((X_train, pd.get_dummies(X_train[z], prefix = z)[good_cols]), axis = 1)
        X_test = pd.concat((X_test, pd.get_dummies(X_test[z], prefix = z)[good_cols]), axis = 1)
        del X_test[z]
        del X_train[z]
    return X_train, X_test



def construction(X_train, X_test):
    for i in [X_train, X_test]:
        i['construction_year'].replace(0, X_train[X_train['construction_year'] != 0]['construction_year'].median(), inplace=True)
    return X_train, X_test


def bools(X_train, X_test):
    """
    public_meeting: we will fill the nulls as 'False'
    permit: we will fill the nulls as 'False
    """
    z = ['public_meeting', 'permit']
    for i in z:
        X_train[i].fillna(False, inplace = True)
        X_train[i] = X_train[i].apply(lambda x: float(x))
        X_test[i].fillna(False, inplace = True)
        X_test[i] = X_test[i].apply(lambda x: float(x))
    return X_train, X_test


def locs(X_train, X_test):
    """
    fill in the nulls for ['longitude', 'latitude', 'gps_height', 'population'] by using means from 
    ['subvillage', 'district_code', 'basin'], and lastly the overall mean
    """
    trans = ['longitude', 'latitude', 'gps_height', 'population']
    for i in [X_train, X_test]:
        i.loc[i.longitude == 0, 'latitude'] = 0
    for z in trans:
        for i in [X_train, X_test]:
            i[z].replace(0., np.NaN, inplace = True)
            i[z].replace(1., np.NaN, inplace = True)
        
        for j in ['subvillage', 'district_code', 'basin']:
        
            X_train['mean'] = X_train.groupby([j])[z].transform('mean')
            X_train[z] = X_train[z].fillna(X_train['mean'])
            o = X_train.groupby([j])[z].mean()
            fill = pd.merge(X_test, pd.DataFrame(o), left_on=[j], right_index=True, how='left').iloc[:,-1]
            X_test[z] = X_test[z].fillna(fill)
        
        X_train[z] = X_train[z].fillna(X_train[z].mean())
        X_test[z] = X_test[z].fillna(X_train[z].mean())
        del X_train['mean']
    return X_train, X_test


def removal2(X_train, X_test):
    z = ['amount_tsh',  'num_private', 'region', 
          'quantity', 'quality_group', 'source_type', 'payment', 
          'waterpoint_type_group',
         'extraction_type_group' , 'scheme_name']
    for i in z:
        del X_train[i]
        del X_test[i]
    return X_train, X_test



def small_n2(X_train, X_test):
    cols = [i for i in X_train.columns if type(X_train[i].iloc[0]) == str]
    #print(cols)
    X_train[cols] = X_train[cols].where(X_train[cols].apply(lambda x: x.map(x.value_counts())) > 100, "other")
    for column in cols:
        for i in X_test[column].unique():
            if i not in X_train[column].unique():
                X_test[column].replace(i, 'other', inplace=True)
    return X_train, X_test




def dummies(X_train, X_test):
    columns = [i for i in X_train.columns if type(X_train[i].iloc[0]) == str]
    for column in columns:
        X_train[column].fillna('NULL', inplace = True)
        good_cols = [column+'_'+i for i in X_train[column].unique() if i in X_test[column].unique()]
        X_train = pd.concat((X_train, pd.get_dummies(X_train[column], prefix = column)[good_cols]), axis = 1)
        X_test = pd.concat((X_test, pd.get_dummies(X_test[column], prefix = column)[good_cols]), axis = 1)
        del X_train[column]
        del X_test[column]
    return X_train, X_test


def lda(X_train, X_test, y_train, cols=['population', 'gps_height', 'latitude', 'longitude']):
    sc = StandardScaler()
    X_train_std = sc.fit_transform(X_train[cols])
    X_test_std = sc.transform(X_test[cols])
    lda = LDA(n_components=None)
    X_train_lda = lda.fit_transform(X_train_std, y_train.values.ravel())
    X_test_lda = lda.transform(X_test_std)
    
    X_train_lda = pd.DataFrame(X_train_lda , index = X_train.index)
    X_test_lda = pd.DataFrame(X_test_lda , index = X_test.index)
    
    X_train = pd.merge(X_train_lda , X_train , left_index=True , right_index = True)
    X_test  = pd.merge(X_test_lda , X_test , left_index=True , right_index=True)
    
    #X_train = pd.concat((pd.DataFrame(X_train_lda), X_train), axis=1)
    #X_test = pd.concat((pd.DataFrame(X_test_lda), X_test), axis=1)
    for i in cols:
        del X_train[i]
        del X_test[i]
    return X_train, X_test

def gini(p):
    return 1-(p**2 + (1-p)**2)

def impurity(X_train):
    imp = {}
    for i in X_train.columns[17:]:
        imp[i] = gini(X_train[i].mean())
    return imp


Created ordinal data feature and few month, yearly features. Missing value imputation have been taken care separately for separate features. 
For location features such as latitude , longitude , gps height and population , use mean value of sub village, district code and basin and finally the overall mean. 
For boolean features : public meeting and permit, fill null values with False.


In [7]:
X_train, X_test = dates(X_train, X_test)
X_train, X_test = dates2(X_train, X_test)

X_train, X_test = construction(X_train, X_test)
X_train, X_test = bools(X_train, X_test)  

In [8]:
X_train, X_test = locs(X_train, X_test)
X_train['population'] = np.log(X_train['population'])
X_test['population'] = np.log(X_test['population'])

We are removing features which are mostly empty or constant- amount_tsh , num_private , scheme_name. Also for highly correlated features we are keeping only 1 feature ( refer EDA analysis).

Also for some of the categorical features ,we are clubbing very infrequent catergories( with count less than 100) into  a single catergory. This will be helpful to control size of one hot encode vectors as well.

In [9]:
X_train, X_test = removal2(X_train, X_test)
X_train, X_test = small_n2(X_train, X_test)

For numerical columns we are converting the features into LDA components

In [10]:
print(X_train.shape)
print(X_test.shape)

(50000, 45)
(5083, 45)


In [11]:
X_train, X_test = lda(X_train, X_test, y_train)
X_train, X_test = dummies(X_train, X_test)
print(X_train.shape)
print(X_test.shape)

(50000, 442)
(5083, 442)


In [15]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [16]:
print(X_tr.shape)
print(X_val.shape)
print(y_tr.shape)
print(y_val.shape)

(37500, 442)
(12500, 442)
(37500, 1)
(12500, 1)


In [18]:
X_tr.head(2)

,0,date_recorded,region_code,district_code,public_meeting,permit,construction_year,month_recorded_2,month_recorded_1,month_recorded_3,...,source_other,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_hand pump,waterpoint_type_communal standpipe,waterpoint_type_other,waterpoint_type_communal standpipe multiple,waterpoint_type_improved spring,waterpoint_type_cattle trough
new_ids,,,,,,,,,,,,,,,,,,,,,
43793,-0.087682,734351,12,4,1.0,0.0,2000,0,0,0,...,0,1,0,0,0,1,0,0,0,0
52729,-1.477396,734335,15,2,1.0,0.0,1991,0,0,0,...,0,1,0,0,1,0,0,0,0,0


The custom metric for data challenge is as follows: false negative rate is weighted 90% more that false positive rate for calcuating error.

In [19]:
def custom_accuracy(y , ypred):
    #set_trace()
    
    misses = 1- (ypred ==y)
    false_pos = misses*(1-y)
    false_neg = misses *y
    false_pos_rate = false_pos.sum()/ (len(y) - int(y.sum()))
    false_neg_rate = false_neg.sum()/ (int(y.sum()))
    
    score = 1 - ( 0.9 * false_neg_rate  +  0.1 * false_pos_rate) 
    return score

In [20]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)
X_val = scaler.transform(X_val)

/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


### Modelling excercise

In [21]:
param_grid = {
             "criterion": [ "entropy" ],  
             'n_jobs' : [3],
             'bootstrap': [True , False], 
             'max_depth': [20, 40],  
             'max_features': ['auto'],
             'min_samples_leaf': [ 2, 4],
             'min_samples_split': [ 2 ,6],
             'n_estimators': [  200]  ,
            'class_weight' : ['balanced']
             }


from sklearn.model_selection import KFold
cv_5 = KFold(n_splits=5)

gs = GridSearchCV(RandomForestClassifier(random_state = 12),param_grid=param_grid,
                scoring  = make_scorer(custom_accuracy),cv= cv_5,  n_jobs=3 , verbose = 5 , refit= True)  #n_iter = 100,

gs = gs.fit(X_tr, y_tr.values.ravel())


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   35.5s
/home/FRACTAL/jitesh.arora/anaconda3/envs/fastainew/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  3.9min
[Parallel(n_jobs=3)]: Done  80 out of  80 | elapsed:  5.1min finished


In [22]:
clf = gs.best_estimator_
clf.fit(X_tr , y_tr.values.ravel());

In [23]:
val_pred = clf.predict(X_val)
print('Custom accuracy in validation set' , custom_accuracy(y_val.values.ravel() , val_pred))

Custom accuracy in validation set 0.8043335824159802


In [45]:
from sklearn.model_selection import StratifiedKFold
cv_5 = StratifiedKFold(n_splits=5)

full_train= X_train.copy()
full_labels = y_train.copy()

Observing custom accuracy across different grid search cross validation data sets. All of them are around 80-81%

In [46]:
for train_ind, test_ind in cv_5.split(full_train, full_labels):

    feat_train, feat_test = full_train.iloc[train_ind], full_train.iloc[test_ind]
    label_train, label_test = full_labels.iloc[train_ind],full_labels.iloc[test_ind]
    clf.fit(feat_train,label_train.values.ravel())
    predictions = gs.best_estimator_.predict_proba(feat_test)
    preds = np.where(predictions[:, 1]>=0.5, 1, 0) 
    print('custom accuracy ' , custom_accuracy(label_test['defective'] , preds))



custom accuracy  0.8085153387273979
custom accuracy  0.8121529442425978
custom accuracy  0.809849756736739
custom accuracy  0.8023772326602174
custom accuracy  0.8061448153562172


### Feature Importance

In [47]:
feature_importance = pd.concat((pd.DataFrame(X_train.columns, columns = ['variable']), 
           pd.DataFrame(clf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

In [48]:
feature_importance

,variable,importance
423,quantity_group_dry,0.110352
0,0,0.062053
6,construction_year,0.048599
1,date_recorded,0.044175
419,quantity_group_enough,0.038828
438,waterpoint_type_other,0.033844
386,extraction_type_class_other,0.029707
378,extraction_type_other,0.026450
2,region_code,0.020152
3,district_code,0.019111


quality_group_dry is coming top most important feature in determining the functional condition of hand pumps. LDA variable with information about population , latitude and longitude is coming next most important. 

We need to look at some algorithms such as LightGBM and CatBoost to take care of categorical features in better ways.
